# 선형 회귀식의 계수를 찾는 법 - OLS VS. SGD
- 보스턴 집값 데이터 활용(RM VS Price)

### 필요한 모듈 import

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 필요한 라이브러리 import 
from sklearn.datasets import load_boston
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


# 1. LinearRegression 모델을 사용한 경우

In [2]:
from sklearn.datasets import load_boston

# 데이터 수집
boston = load_boston()

import pandas as pd

df =pd.DataFrame(boston.data, columns=boston.feature_names)

X = pd.DataFrame(df['RM'])
y = boston.target

# 전체 데이터 중 80%는 학습용, 20%는 검증용으로 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,\
                                                   random_state=1)
import numpy as np
from sklearn.linear_model import LinearRegression

#모델 객체 생성
reg = LinearRegression()

# 모델 학습
reg.fit(X_train, y_train)

# 계수 및 절편 확인: _속성은 학습을 통해 결정되는 속성
print(reg.coef_, reg.intercept_)

#회귀식
print("y = {:2f}X + {:.3f}".format(reg.coef_[0], reg.intercept_))

from sklearn.metrics import mean_squared_error, r2_score

# 예측 수행
y_pred = reg.predict(X_test)

# MSE, RMSE, r2_score
mse = mean_squared_error(y_test, y_pred) # 평균제곱오차
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred) # 결정계수 

print("MSE:", np.round(mse, 3))
print("RMSE: ", np.round(rmse, 3))
print("R2: ", np.round(r2, 3))

[8.46109164] -30.571032410898315
y = 8.461092X + -30.571
MSE: 36.517
RMSE:  6.043
R2:  0.602


# 2. SGDRegressor with Hyperparameter
#### tol을 작게하니까 점수 더 올라간다.

In [3]:
from sklearn.linear_model import SGDRegressor

reg = SGDRegressor(max_iter=10000000, eta0 = 0.01,tol = 0.0001, learning_rate = 'constant')

# 모델 학습
reg.fit(X_train, y_train)

# 계수 및 절편 확인: _속성은 학습을 통해 결정되는 속성
print(reg.coef_, reg.intercept_)

#회귀식
print("y = {:2f}X + {:.3f}".format(reg.coef_[0], reg.intercept_[0])) # 둘다 넘파이니까 절편에도 0을 지정해주어야 한다. 

from sklearn.metrics import mean_squared_error, r2_score

# 예측 수행
y_pred = reg.predict(X_test)

# MSE, RMSE, r2_score
mse = mean_squared_error(y_test, y_pred) # 평균제곱오차
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred) # 결정계수 

print("MSE:", np.round(mse, 3))
print("RMSE: ", np.round(rmse, 3))
print("R2: ", np.round(r2, 3))

[6.39639752] [-17.32315912]
y = 6.396398X + -17.323
MSE: 43.212
RMSE:  6.574
R2:  0.529


SGD는 임의의 시작점이기 때문에 R2 값이 모두 다르다. 
하이퍼파라미터(학습률, 에폭)... 

###### tol, max_iter, learning_rate eta0: 첫번째 기울기에 곱해지는 .. 
###### 러닝레이트에 들어가는 인자들은 첫 스텝을 내려왔을 때 사용한 학습률을 다 내려갈때까지 동일하게 곱해줄건지 
###### 내려가면서 학습률 다르게할건지 
###### 우리는 계속 마지막 스텝까지 곱해주는 학습률을 유지한다는 의미에서 constant 설정

In [4]:
help(SGDRegressor) # 인자 정보 볼 수 있음 

Help on class SGDRegressor in module sklearn.linear_model._stochastic_gradient:

class SGDRegressor(BaseSGDRegressor)
 |  SGDRegressor(loss='squared_loss', *, penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, random_state=None, learning_rate='invscaling', eta0=0.01, power_t=0.25, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, warm_start=False, average=False)
 |  
 |  Linear model fitted by minimizing a regularized empirical loss with SGD
 |  
 |  SGD stands for Stochastic Gradient Descent: the gradient of the loss is
 |  estimated each sample at a time and the model is updated along the way with
 |  a decreasing strength schedule (aka learning rate).
 |  
 |  The regularizer is a penalty added to the loss function that shrinks model
 |  parameters towards the zero vector using either the squared euclidean norm
 |  L2 or the absolute norm L1 or a combination of both (Elastic Net). If the
 | 

### 각각의 점에서 기울기를 계산하는 것이었기 때문에 특성값이 동일한 범위 내에 있어야 한다. 
##### = 어떤 것은 특성이 방개수, 크기, 역세권 (3, 34, 500) : 각각 단위차이가 많이 난다. 
##### 이 경우엔  계산이 느려지고 정확하지 않다. 
##### 때문에 값을 변환시켜준다. 
##### 변환 중에 각각 서로다른 범위를 동일한 범위로 
##### -> MinMax 
##### -> Standard 표준화 해당 값에서 평균을 빼고 표준편차로 나눠서 가져오도록 평균 0. 분산 1 
##### 경사하강법을 쓸 때에는 각각의 특성이 비슷한 범위안에 들어올 수 있도록 변환시켜주는 것이 중요하다

# 3. SGDRegressor with Scaling
#### X의 입력값을 스케일링해서 쓸 것이다


In [5]:
from sklearn.linear_model import SGDRegressor

# 표준화 스케일링 
train_mean = np.mean(X_train, axis = 0)
train_std = np.std(X_train, axis=0)
X_train = (X_train - train_mean) / train_std 
X_test = (X_test - train_mean) / train_std # 훈련데이터를 기준으로 ... 


reg = SGDRegressor(max_iter=1000, eta0 = 0.01, learning_rate = 'constant')

# 모델 학습
reg.fit(X_train, y_train)

# 계수 및 절편 확인: _속성은 학습을 통해 결정되는 속성
print(reg.coef_, reg.intercept_)

#회귀식
print("y = {:2f}X + {:.3f}".format(reg.coef_[0], reg.intercept_[0])) # 둘다 넘파이니까 절편에도 0을 지정해주어야 한다. 

from sklearn.metrics import mean_squared_error, r2_score

# 예측 수행
y_pred = reg.predict(X_test)

# MSE, RMSE, r2_score
mse = mean_squared_error(y_test, y_pred) # 평균제곱오차
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred) # 결정계수 

print("MSE:", np.round(mse, 3))
print("RMSE: ", np.round(rmse, 3))
print("R2: ", np.round(r2, 3))

[5.23250227] [21.94110531]
y = 5.232502X + 21.941
MSE: 38.858
RMSE:  6.234
R2:  0.576


###### 가장 적합한 하이퍼파라미터를 찾아내는 것 중요하다. 
###### 그런데 일일이 하는 것 힘드니까 하이퍼파라미터를 범위 내에서 다 주고 기계가 돌리도록 한다. 
###### 표준화도 분명히 효과가 있기 때문에 중요하다. 


# 4. SGDRegressor with StandardScaler()
#### 아예 사이킷런에서 스케일링을 제공하고 있기 때문에 이것을 활용해보자

In [6]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

# 표준화 스케일링 
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train) # 이전 코드 대신임 
X_test = scaler.transform(X_test)
# x-train은 모델의 기준을 만드는데 쓰고 검증 데이터는 만들어진 기준에 transform만 쓴다. 
# 때문에 예측값을 만들려고 쓰는 데이터는 무조건 fit 안함 (당연하다. 테스트데이터로 왜 학습모델을 만드나)
# y는 종속변수(알아내고싶은 값)이기 때문에 y는 고치지 않는다


reg = SGDRegressor(max_iter=1000, eta0 = 0.01, learning_rate = 'constant')

# 모델 학습
reg.fit(X_train, y_train)

# 계수 및 절편 확인: _속성은 학습을 통해 결정되는 속성
print(reg.coef_, reg.intercept_)

#회귀식
print("y = {:2f}X + {:.3f}".format(reg.coef_[0], reg.intercept_[0])) # 둘다 넘파이니까 절편에도 0을 지정해주어야 한다. 

from sklearn.metrics import mean_squared_error, r2_score

# 예측 수행
y_pred = reg.predict(X_test)

# MSE, RMSE, r2_score
mse = mean_squared_error(y_test, y_pred) # 평균제곱오차
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred) # 결정계수 

print("MSE:", np.round(mse, 3))
print("RMSE: ", np.round(rmse, 3))
print("R2: ", np.round(r2, 3))

[6.81415512] [22.13985474]
y = 6.814155X + 22.140
MSE: 34.525
RMSE:  5.876
R2:  0.623


#### OLS에서 긋는 회귀선과 (방정식을 풀어서 나온 값)
##### 조금씩 최적값으로 이동하면서 얻는 근사치인데 이를 잘 조정하면 정규방정식의 해를 찾아갈 수 있다. 